In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import prophet
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json
import datetime
import random

In [2]:
dataset_path = '../data/'

pollen_test = pd.read_csv(dataset_path + 'pollen_test.csv')
pollen_train = pd.read_csv(dataset_path + 'pollen_train.csv')
pollen_type = pd.read_csv(dataset_path + 'pollen-type.csv')
submission_example = pd.read_csv(dataset_path + 'submission_example.csv')
weather_data = pd.read_csv(dataset_path + 'weather_data.csv')

In [3]:
with open('../results/prophet_by_town.csv', 'w') as f:
    f.write('batch_id,1 day prediction,2 days prediction,3 days prediction\n')

In [4]:
print(max(pollen_test.AMBROSIA))

458


In [5]:
pollen_test.head()

,Unnamed: 0,location,date,ACER,ALNUS,AMBROSIA,ARTEMISIA,BETULA,CANNABACEAE,CARPINUS,...,PLATANUS,POACEAE,POPULUS,QUERCUS,RUMEX,SALIX,TILIA,ULMACEAE,URTICACEAE,batch_id
0,0,БЕОГРАД - НОВИ БЕОГРАД,2022-02-14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,БЕОГРАД - НОВИ БЕОГРАД,2022-02-15,0,5,0,0,6,0,0,...,0,0,6,0,0,0,0,13,0,1
2,2,БЕОГРАД - НОВИ БЕОГРАД,2022-02-16,0,0,0,0,1,0,0,...,0,0,6,0,0,0,0,13,0,1
3,3,БЕОГРАД - НОВИ БЕОГРАД,2022-02-17,0,6,0,0,11,0,0,...,0,0,22,0,0,0,0,30,0,1
4,4,БЕОГРАД - НОВИ БЕОГРАД,2022-02-18,0,12,0,0,2,0,0,...,0,0,47,0,0,0,0,58,0,1


In [6]:
pollen_df = pollen_test.copy()

pollen_df['date'] = pd.to_datetime(pollen_df['date'])

# change the column names
pollen_df['ds'] = pollen_df['date']
pollen_df['y'] = pollen_df['AMBROSIA']
pollen_df['floor'] = 0
pollen_df['cap'] = 2500

pollen_df.drop(['Unnamed: 0', 'date', 'AMBROSIA'], axis=1, inplace=True)

pollen_df.tail()

,location,ACER,ALNUS,ARTEMISIA,BETULA,CANNABACEAE,CARPINUS,CELTIS,CHENOP/AMAR.,CORYLUS,...,RUMEX,SALIX,TILIA,ULMACEAE,URTICACEAE,batch_id,ds,y,floor,cap
1115,СУБОТИЦА,0,0,0,0,3,0,0,0,0,...,0,0,0,0,2,112,2022-10-15,4,0,2500
1116,СУБОТИЦА,0,0,1,0,1,0,0,2,0,...,0,0,0,0,0,112,2022-10-16,3,0,2500
1117,СУБОТИЦА,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-17,2,0,2500
1118,СУБОТИЦА,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-18,7,0,2500
1119,СУБОТИЦА,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,112,2022-10-19,2,0,2500


In [7]:
def next_three_days(x):
    return [x + datetime.timedelta(days=1), x + datetime.timedelta(days=2), x + datetime.timedelta(days=3)]

def predict_for_dates(m, dates, ratio):
    future = pd.DataFrame({'ds': dates})
    future['floor'] = 0
    future['cap'] = 2500
    
    forecast = m.predict(future)
    ans = []
    for forecast_row in forecast.itertuples():
        val = int(ratio * forecast_row.yhat)
        val = max(0, val)
        ans.append(val)

        '''
        if forecast_row.yhat_lower < 0:
            ans.append(0)
        else:
            # pick random value between yhat_lower and yhat_upper
            # ans.append(random.randrange(int(forecast_row.yhat_lower), int(forecast_row.yhat_upper)))
            val = int(ratio * random.randrange(int(forecast_row.yhat_lower), int(forecast_row.yhat_upper)))
            print(val, ratio, forecast_row.yhat, forecast_row.yhat_lower, forecast_row.yhat_upper)
            ans.append(val)
        '''

    return ans

## Start Test

In [8]:
BATCH_SIZE = 113
for batch_id in range(1, BATCH_SIZE):
    print('--------- Batch id: ' + str(batch_id) + ' ---------')

    # extract pollen_df where column batch_id == batch_id
    test_df = pollen_df[pollen_df['batch_id'] == batch_id]
    # Drop index for test_df
    test_df.reset_index(inplace=True, drop=True)

    with open('../models/prophet/' + test_df['location'][0] + '_prophet.json', 'r') as fin:
        m = model_from_json(fin.read())  # Load model

    forecast = m.predict(test_df)
    forecast['yhat'] = [max(0, x) for x in forecast['yhat']]
    
    # Make a list of difference between yhat and y
    max_real_val = max(test_df['y'])
    max_pred_val = max(forecast['yhat'])

    ratio = np.mean([max_real_val / max_pred_val if max_pred_val != 0 else 0 for i in range(len(forecast))])
    
    print(ratio)

    # 3 days after test_df
    last_date = test_df['ds'].iloc[-1]
    predict_dates = next_three_days(last_date)

    results = predict_for_dates(m, predict_dates, ratio)

    print(last_date, list(test_df['y']), list(forecast['yhat']))
    print([int(ratio * x) for x in forecast['yhat']], results)

    # Save results to result.csv
    with open('../results/prophet_by_town.csv', 'a') as fout:
        results = str(batch_id) + ',' + ','.join([str(x) for x in results]) + '\n'
        fout.write(results)

--------- Batch id: 1 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.0
2022-02-23 00:00:00 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [7.6624736402982805, 4.085225701450664, 3.4192989522330492, 6.2401458865558865, 7.327680661635849, 2.711750229615429, 3.176702838437265, 9.012371134345226, 5.779000509861458, 5.399790079868968]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0]
--------- Batch id: 2 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.0
2022-02-23 00:00:00 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.0, 3.5601675030966717, 1.623464223172256, 0.7048232944600912, 0.5272588582490734, 5.734537397689326, 5.229401912493042, 4.154398056989589, 9.818276972781074, 7.909209227632491]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0]
--------- Batch id: 3 ---------


...

0.0
2022-10-19 00:00:00 [10, 3, 0, 3, 0, 0, 11, 3, 3, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0]
--------- Batch id: 111 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.48850695581041637
2022-10-19 00:00:00 [7, 3, 2, 5, 4, 8, 2, 4, 1, 3] [14.08800370038456, 14.456078284231419, 8.624837008887951, 16.12245591155691, 16.376430068898966, 15.98042277674014, 14.144918686338016, 16.169011023655674, 14.090886062437018, 5.871392228784041]
[6, 7, 4, 7, 8, 7, 6, 7, 6, 2] [5, 4, 3]
--------- Batch id: 112 ---------


c:\Users\Branko\miniconda3\lib\site-packages\prophet\serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


0.27611615924671085
2022-10-19 00:00:00 [6, 2, 6, 5, 3, 4, 3, 2, 7, 2] [22.96537518647184, 22.3784243455721, 21.61602825275999, 22.545305925556512, 25.351649172207534, 24.944599769885933, 24.291066498588908, 18.121448513778624, 14.057703416125783, 10.141317636010317]
[6, 6, 5, 6, 7, 6, 6, 5, 3, 2] [2, 2, 1]
